In [1]:
## library imports

## db connection params
from config import get_engine

## IO
import pandas as pd

## linear algebra
import numpy as np

## defaultdict
from collections import defaultdict

In [2]:
## db engine

engine = get_engine("processingdev")

In [120]:
## scores

scores = pd.read_sql("SELECT * FROM Scores LIMIT 10000;",con=engine)
scores.head()

,GameID,EventID,EventName,Draw,End1,End2,End3,End4,End5,End6,...,Hammer,FinalScore,Team,URL,DrawNum,Year,StartMonth,EndMonth,EventDayStart,EventDayEnd
0,1,3000,Swiss Curling League,1,0.0,0.0,1.0,0.0,1.0,0.0,...,0,3,Marc Suter,https://www.curlingzone.com/event.php?eventid=...,1,2013,Jan,Feb,11,3
1,1,3000,Swiss Curling League,1,0.0,2.0,0.0,3.0,0.0,2.0,...,1,9,Peter de Cruz,https://www.curlingzone.com/event.php?eventid=...,1,2013,Jan,Feb,11,3
2,2,3000,Swiss Curling League,1,2.0,0.0,0.0,0.0,0.0,0.0,...,1,7,Sven Michel,https://www.curlingzone.com/event.php?eventid=...,1,2013,Jan,Feb,11,3
3,2,3000,Swiss Curling League,1,0.0,0.0,2.0,1.0,0.0,0.0,...,0,5,Alexander Attinger,https://www.curlingzone.com/event.php?eventid=...,1,2013,Jan,Feb,11,3
4,3,3000,Swiss Curling League,1,1.0,0.0,3.0,0.0,1.0,2.0,...,1,11,St. Gallen Bar,https://www.curlingzone.com/event.php?eventid=...,1,2013,Jan,Feb,11,3


In [121]:
## class code

class Hammer(object):
    def __init__(self,hammer):
        
        ## check if two teams are passed
        assert len(hammer.items()) == 2
        
        keys,values = tuple(hammer.keys()), tuple(hammer.values())
        
        
        ## check team names are different
        assert values[0] != values[1]
        
        ## check 1 and 0 is passed
        assert not all(keys)
        
        
        self.hammer = hammer.copy()
        
    def get_hammer(self):
        return self.hammer.copy()


    def get_next_hammer(self,EndResult):
        
        
        
        end_result = EndResult.get_end_result()
        current_hammer = self.get_hammer()
        
        ## check teams match up
        
        hammer_teams = [team for team in current_hammer.values()]
        end_result_teams = [team for team in end_result.keys()]
        
        ## I don't think I have to check the opposite 
        assert all(
        
            [team in end_result_teams for team in hammer_teams]
        
        
        )
        
        next_hammer = {}
        
        if end_result[current_hammer[1]] > 0:
            next_hammer[0] = current_hammer[1]
            next_hammer[1] = current_hammer[0]
        else:
            next_hammer[1] = current_hammer[1]
            next_hammer[0] = current_hammer[0]
            
            
        return Hammer(next_hammer)
    
    def to_frame(self):
        hammer = self.get_hammer()
        columns = ['Hammer','Team']
        situation = list(hammer.items())
        
        return pd.DataFrame(data=situation,columns=columns)
    
    
class EndResult(object):
    def __init__(self,end_result):
        
        
        ## check two results (1 for each team) are passed
        assert len(end_result.items()) == 2
    
        self.end_result = end_result.copy()
        
    def get_end_result(self):
        return self.end_result.copy()
    
    
    def to_frame(self):
        end_result = self.get_end_result()
        
        
        columns = ['Team','EndResult']
        results = [[team,result] for team,result in end_result.items()]
        
        if results[0][1] > results[1][1]:
            results[1][1] = results[0][1]*-1
        else:
            results[0][1] = results[1][1]*-1
            
    
        return pd.DataFrame(data=results,columns=columns)
    
class Score(object):
    def __init__(self,score):
        
        ## check two scores (1 for each team) is passed
        assert len(score.items()) == 2
        
        teams,scores = tuple(score.keys()),tuple(score.values())
        
        
        ## check scores are higher than 0 
        assert all(
        
            [_score >= 0 for _score in scores]
        
        )
        
        self.score = score.copy()
        
    def get_score(self):
        return self.score.copy()
    
    def to_frame(self):
        
        score = self.get_score()
        
        columns = ['Team','Score']
        situation = list(score.items())
        
        return pd.DataFrame(data=situation,columns=columns)
    
class End(object):
    def __init__(self,EndScore,EndHammer,EndResult = None,end_num=1):
        self.end_num = end_num
        self.end_score_obj = EndScore
        self.end_hammer_obj = EndHammer
        self.end_result_obj = EndResult
    
    
    
    def get_end_num(self):
        return self.end_num
    
    def get_end_score_obj(self):
        return self.end_score_obj
    
    def get_end_hammer_obj(self):
        return self.end_hammer_obj
    
    def get_end_result_obj(self):
        return self.end_result_obj
    
    def get_next_end(self,EndResult):
        end_result_obj = EndResult
        end_result = end_result_obj.get_end_result()
        
        end_score_obj = self.get_end_score_obj()
        end_score = end_score_obj.get_score()
        
        end_hammer_obj = self.get_end_hammer_obj()
        next_hammer_obj = end_hammer_obj.get_next_hammer(end_result_obj)
        
        for team in end_score.keys():
            end_score[team] += end_result[team]
            
        next_score_obj = Score(end_score)
            
        self.end_result_obj = EndResult
        
        end_num = self.get_end_num()
        next_end_num = end_num + 1
        
        return End(EndScore = next_score_obj,EndHammer = next_hammer_obj,EndResult=None,end_num=next_end_num)
    
    def to_frame(self):
        hammer_obj = self.get_end_hammer_obj()
        end_score_obj = self.get_end_score_obj()
        
        
        df = hammer_obj.to_frame()
        df['End'] = self.get_end_num()
        
        end_result_obj = self.get_end_result_obj()
        
        if end_result_obj:
            df = df.merge(end_score_obj.to_frame(),on='Team',how='inner').merge(end_result_obj.to_frame(),on='Team',how='inner')
        else:
            df = df.merge(end_score_obj.to_frame(),on='Team',how='inner')
            df['EndResult'] = np.nan
            
        return df[['Team','End','Hammer','Score','EndResult']]
        
        
class Game(object):
    def __init__(self,teams,game_id,final_score=None):
        
        ## check two teams are in the passed list
        assert len(teams) == 2,"Invalid number of teams passed"
        
        ## check only distinct team names have been passed
        team_dict = defaultdict(int)
        
        for team in teams:
            team_dict[team] +=1
            
        assert all(
        
            [count == 1 for count in team_dict.values()]
        
        
        )
        
        
        self.teams = teams.copy()
        
        self.game_id = game_id
        
        self.ends = []
        
        if final_score:
            
            score = final_score.get_score()
            score_teams = score.keys()
            
            ## I don't think I have to check the opposite 
            assert all(
        
            [team in score_teams for team in teams]
        
            )
        
        
            self.final_score = final_score
        else:
            self.final_score = final_score
        
    def get_ends(self):
        return self.ends.copy()
    def get_game_id(self):
        return self.game_id
        
    def add_end(self,End):
        ends = self.get_ends()
        ends.append(End)
        self.ends = ends.copy()
        
    def to_frame(self):
        df = pd.concat([end.to_frame() for end in self.get_ends()],axis=0,sort=True).reset_index(drop=True)
        df['GameID'] = self.get_game_id()
        return df

In [122]:
def create_game(game_id,games):
    ends = [1,2,3,4,5,6,7,8,9,10,11,12]
    ends = list(map(str,ends))
    
    df = games.loc[games.GameID == game_id]
    assert len(df) == 2
    
    teams = []
    final_score = {}
    hammer_situation = {}
    
    for score,team,hammer in df[['FinalScore','Team','Hammer']].values:
        teams.append(team)
        final_score[team] = score
        hammer_situation[hammer] = team
        
    final_score_obj = Score(final_score)
    hammer_obj = Hammer(hammer_situation)
    game_obj = Game(teams=teams,game_id = game_id,final_score = final_score_obj)
    end = End(EndScore=Score({team:0 for team in teams}),EndHammer = hammer_obj)
    
    
    for e in ends:
        if end.get_end_score_obj().get_score() == final_score_obj.get_score():
            break
        result = EndResult({team:score for score,team in df[["End{}".format(e),'Team']].values})
        game_obj.add_end(end)
        end = end.get_next_end(EndResult=result)
        
    return game_obj 
            